In [8]:
import os
import pandas as pd
import numpy as np

In [9]:
extract_path='C4MIP_data/c4mip_cmip6_ascii_files_11_models'

def process_variable_data(file_path, column_name):
    try:
        df = pd.read_csv(file_path, delim_whitespace=True, index_col=0, engine='python')
        if column_name in df.columns:
            return df[column_name]
        else:
            return None
    except Exception as e:
        return None

def round_down_index(df):
    df.index = df.index.astype(int)
    return df.groupby(level=0).first()

variables = ['CO2', 'LA_CO2_FLUX', 'OA_CO2_FLUX', 'ST']
data_frames = {var: {} for var in variables}

for root, dirs, files in os.walk(extract_path):
    for file in files:
        if file.endswith('_COU.txt'):
            model_name = file.split('_')[0]
            print(model_name)
            file_path = os.path.join(root, file)
            for var in variables:
                var_column = process_variable_data(file_path, var)

                if var_column is not None:
                    if model_name in data_frames[var]:
                        data_frames[var][model_name] = pd.concat([data_frames[var][model_name], var_column], axis=1).mean(axis=1)
                    else:
                        data_frames[var][model_name] = var_column
                else:
                    print('Variable {} not found in {}'.format(var, file_path))

dfs = {var: round_down_index(pd.DataFrame(data_frames[var]).groupby(level=0).first()) for var in variables}

df_oa=dfs['OA_CO2_FLUX']
df_la=dfs['LA_CO2_FLUX']
df_co2=dfs['CO2']
df_tas=dfs['ST']
scon=np.sign(df_oa.cumsum().iloc[100])
df_oa = df_oa * scon
df_la = df_la * scon
c_emis=(((df_co2-df_co2.iloc[0]).diff()*2.13)+df_la+df_oa)


c_emis.index=c_emis.index-1850
df_tas.index=df_tas.index-1850
df_tas=df_tas-(2*df_tas.iloc[:30].mean()-df_tas.iloc[15:45].mean())

mdls1=c_emis.columns
ce_1000=c_emis.copy()
for m in mdls1:
    ce_1000.loc[ce_1000[m].cumsum()>1000,m]=np.nan
mdlmap={}
mdlmap['ACCESS-ESM1-5']='ACCESS-ESM1.5'
mdlmap['CESM2']='CESM2'
mdlmap['CNRM-ESM2-1']='CNRM-ESM2.1'
mdlmap['GFDL-ESM4']='GFDL'
mdlmap['MPI-ESM1-2-LR']='MPIESM'
mdlmap['NorESM2-LM']='NorESM2LM'
mdlmap['UKESM']='UKESM'
mdlmap['UVic-ESCM-2-10']='UVic-ESCM-2-10'

usmdls=list(mdlmap.keys())

inv_map = {v: k for k, v in mdlmap.items()}



CESM2
ACCESS-ESM1.5
GFDL
NorESM2LM
CanESM5
BCC
UKESM
CNRM-ESM2.1
JAMSTEC
MPIESM
IPSL


/var/folders/pd/218my53900s1gy0k6lcyvjxm0000gn/T/ipykernel_32248/2223730113.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True, index_col=0, engine='python')
/var/folders/pd/218my53900s1gy0k6lcyvjxm0000gn/T/ipykernel_32248/2223730113.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True, index_col=0, engine='python')
/var/folders/pd/218my53900s1gy0k6lcyvjxm0000gn/T/ipykernel_32248/2223730113.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True, index_col=0, engine='python')
/var/folders/pd/218my53900s1gy0k6lcyvjxm0000gn/T/ipykernel_32248/2223730113.py:

In [10]:
df_oa=dfs['OA_CO2_FLUX']
df_la=dfs['LA_CO2_FLUX']
df_co2=dfs['CO2']
df_tas=dfs['ST']

In [11]:
scon=np.sign(df_oa.cumsum().iloc[100])
df_oa = df_oa * scon
df_la = df_la * scon

In [22]:
co2tmp=pd.read_csv('C4MIP_data/UVIC/co2_UVicESCM2.10_1pct_bench_1850-1989.csv',index_col=0)
tastmp=pd.read_csv('C4MIP_data/UVIC/tas_UVicESCM2.10_1pct_bench_1850-1989.csv',index_col=0) 
nbptmp=pd.read_csv('C4MIP_data/UVIC/nbp_UVicESCM2.10_1pct_bench_1850-1989.csv',index_col=0)
fgco2tmp=pd.read_csv('C4MIP_data/UVIC/fgco2_UVicESCM2.10_1pct_bench_1850-1989.csv',index_col=0)

In [30]:
fgco2tmp.iloc[:,0].values.shape

(140,)

In [73]:
df_oa.loc[:,'UVic-ESCM-2-10']=pd.Series(fgco2tmp.iloc[:,0].values)

df_la.loc[:,'UVic-ESCM-2-10']=pd.Series(nbptmp.iloc[:,0].values)
df_co2.loc[:,'UVic-ESCM-2-10']=pd.Series(co2tmp.iloc[:,0].values)
df_tas.loc[:,'UVic-ESCM-2-10']=pd.Series(tastmp.iloc[:,0].values)
df_la['UVic-ESCM-2-10'].iloc[:140]=nbptmp.iloc[:,0].values
df_oa['UVic-ESCM-2-10'].iloc[:140]=fgco2tmp.iloc[:,0].values
df_co2['UVic-ESCM-2-10'].iloc[:140]=co2tmp.iloc[:,0].values

df_oa

/var/folders/pd/218my53900s1gy0k6lcyvjxm0000gn/T/ipykernel_32248/585422778.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_la['UVic-ESCM-2-10'].iloc[:140]=nbptmp.iloc[:,0].values
/var/folders/pd/218my53900s1gy0k6lcyvjxm0000gn/T/ipyker

,CESM2,ACCESS-ESM1.5,GFDL,NorESM2LM,CanESM5,BCC,UKESM,CNRM-ESM2.1,JAMSTEC,MPIESM,IPSL,UVic-ESCM-2-10
1850,-0.2414,0.29,0.24,0.14,-0.019056,0.1897,0.363334,-0.821146,0.16,0.10,0.54,-0.081473
1851,0.2677,0.73,0.24,0.50,0.366100,0.5640,0.596087,-0.244671,0.61,0.57,0.80,0.274634
1852,0.5520,0.91,0.74,0.74,0.673920,0.8168,0.900500,-0.036564,0.96,0.66,1.03,0.576484
1853,0.8268,1.02,0.86,0.92,0.965300,1.0387,0.979567,-0.029507,1.08,0.81,1.26,0.828520
1854,0.9538,1.38,1.19,1.21,1.174200,1.3243,1.215028,0.327955,1.22,1.15,1.42,1.049120
...,...,...,...,...,...,...,...,...,...,...,...,...
1997,4.4664,NaN,5.48,NaN,4.533200,6.2508,4.747364,4.174239,4.82,NaN,5.79,NaN
1998,4.5186,NaN,5.40,NaN,4.523200,6.1895,4.731746,4.321155,5.00,NaN,5.53,NaN
1999,4.5933,NaN,5.49,NaN,4.598300,5.9258,4.523351,3.810270,4.84,NaN,5.59,NaN
2000,NaN,NaN,NaN,NaN,4.652800,NaN,4.812630,NaN,NaN,NaN,NaN,NaN


In [74]:
c_emis=(((df_co2-df_co2.iloc[0]).diff()*2.13)+df_la+df_oa)
c_emis


,CESM2,ACCESS-ESM1.5,GFDL,NorESM2LM,CanESM5,BCC,UKESM,CNRM-ESM2.1,JAMSTEC,MPIESM,IPSL,UVic-ESCM-2-10
1850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1851,6.388010,8.2431,2.9769,6.0550,6.22928,7.0120,8.804397,6.160400,5.3405,7.1492,5.2292,5.516479
1852,7.124615,8.3470,6.5872,6.9031,8.33632,8.0395,9.105066,7.468243,7.5431,4.5631,6.8231,6.844646
1853,8.498746,10.7596,5.8550,8.1270,10.34140,7.9381,8.297545,7.373757,8.6483,5.9070,8.4983,7.345543
1854,8.557429,9.9948,7.4792,7.7809,9.37050,8.2849,10.122484,7.922232,8.5709,9.2609,8.8296,7.813020
...,...,...,...,...,...,...,...,...,...,...,...,...
1997,35.143465,NaN,35.2063,NaN,42.76020,38.2865,30.967967,34.670524,34.7321,NaN,30.7060,NaN
1998,36.487886,NaN,35.2268,NaN,42.63920,35.9907,27.946515,34.677869,37.2290,NaN,29.9890,NaN
1999,35.525163,NaN,33.4536,NaN,39.19100,35.0547,33.206343,32.874904,32.8433,NaN,31.9720,NaN
2000,NaN,NaN,NaN,NaN,41.30310,NaN,32.692822,NaN,NaN,NaN,NaN,NaN


In [75]:
df_tas.rename(columns=inv_map).filter(usmdls).to_csv('c4mip_tas.csv')

In [76]:
c_emis.rename(columns=inv_map).filter(usmdls).to_csv('c4mip_emis.csv')